In [1]:
#Starting Spark application

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1687163447754_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
%%info

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
1,application_1687163447754_0002,pyspark,idle,Link,Link,✔


In [3]:
import pandas as pd
import numpy as np
import io
import os
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split
from pyspark.sql.functions import element_at, split, col, pandas_udf, PandasUDFType, udf


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print('Versions: ')
print('numpy: ',np.__version__)
print('pandas: ',pd.__version__)
print('tensorflow: ',tf.__version__)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Versions: 
numpy:  1.16.5
pandas:  1.2.5
tensorflow:  2.4.1

In [5]:
path = 's3://one-image-bucket'#bucket path
path_data= path+'/Test' #path_data
path_result = path+'/Results'#results path
path_result_pca=path+'/ResultsPCA'#PCA results path
print('path:        '+\
      path+'\npath_data:   '+\
      path_data+'\npath_result: '+path_result +\
      '\npath_result_pca: '+path_result_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

path:        s3://one-image-bucket
path_data:   s3://one-image-bucket/Test
path_result: s3://one-image-bucket/Results
path_result_pca: s3://one-image-bucket/ResultsPCA

In [6]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(path_data)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
images.show(5)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------------+------+--------------------+
|                path|   modificationTime|length|             content|
+--------------------+-------------------+------+--------------------+
|s3://one-image-bu...|2023-06-15 08:23:32| 56583|[FF D8 FF E0 00 1...|
|s3://one-image-bu...|2023-06-15 08:23:32| 56054|[FF D8 FF E0 00 1...|
|s3://one-image-bu...|2023-06-15 08:23:32| 56040|[FF D8 FF E0 00 1...|
|s3://one-image-bu...|2023-06-15 08:23:32| 55913|[FF D8 FF E0 00 1...|
|s3://one-image-bu...|2023-06-15 08:23:32| 55693|[FF D8 FF E0 00 1...|
+--------------------+-------------------+------+--------------------+
only showing top 5 rows

In [8]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- modificationTime: timestamp (nullable = true)
 |-- length: long (nullable = true)
 |-- content: binary (nullable = true)
 |-- label: string (nullable = true)

None
+------------------------------------------------+----------+
|path                                            |label     |
+------------------------------------------------+----------+
|s3://one-image-bucket/Test/cucumber_1/r0_199.jpg|cucumber_1|
|s3://one-image-bucket/Test/cucumber_1/r0_175.jpg|cucumber_1|
|s3://one-image-bucket/Test/cucumber_1/r0_163.jpg|cucumber_1|
|s3://one-image-bucket/Test/cucumber_1/r0_3.jpg  |cucumber_1|
|s3://one-image-bucket/Test/cucumber_1/r0_171.jpg|cucumber_1|
+------------------------------------------------+----------+
only showing top 5 rows

None

In [9]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

14540800/14536120 [==============================] - 1s 0us/step

In [10]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
brodcast_weights = sc.broadcast(new_model.get_weights())

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
new_model.summary()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 112, 112, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 112, 112, 32) 128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 112, 112, 32) 0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

In [13]:
#  Returns a MobileNetV2 model with top layer removed and broadcasted pretrained weights.
def model_fn():
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
# Preprocesses raw image bytes for prediction.
def preprocess(content):
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

#  Featurize a pd.Series of raw images using the input model
def featurize_series(model, content_series):
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    output = [p.flatten() for p in preds]#flatten multidemensional preds
    return pd.Series(output)



FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
#This method is a Scalar Iterator pandas UDF wrapping our featurization function.The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).:param content_series_iter: This argument is an iterator over batches of data, where each batchis a pandas Series of image data.
 # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

/usr/lib/spark/python/lib/pyspark.zip/pyspark/sql/pandas/functions.py:392: UserWarning: In Python 3.6+ and Spark 3.0+, it is preferred to specify type hints for pandas UDF instead of specifying pandas UDF type which will be deprecated in the future releases. See SPARK-28264 for more details.

In [16]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
print(path_result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

s3://one-image-bucket/Results

In [18]:
#features_df.write.mode("overwrite").parquet(path_result) #results for CNN classification

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
#Reading results from s3 (format parquet)
if True:
    if True:
        print('lecture distante S3')
        df_parquet = spark.read.format('parquet').load(path_result)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lecture distante S3

In [20]:
df_parquet.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+
|                path|     label|            features|
+--------------------+----------+--------------------+
|s3://one-image-bu...|cucumber_1|[2.020552, 0.1632...|
|s3://one-image-bu...|cucumber_1|[0.676391, 0.1460...|
|s3://one-image-bu...|  carrot_1|[0.33251444, 0.37...|
|s3://one-image-bu...|  carrot_1|[0.3478044, 0.003...|
|s3://one-image-bu...|  carrot_1|[0.0818258, 0.149...|
|s3://one-image-bu...|  carrot_1|[0.19739081, 0.12...|
|s3://one-image-bu...|  carrot_1|[0.720204, 0.2744...|
|s3://one-image-bu...|cucumber_1|[0.7077083, 0.029...|
|s3://one-image-bu...|cucumber_1|[1.429243, 0.0738...|
|s3://one-image-bu...|cucumber_1|[0.7284845, 0.536...|
|s3://one-image-bu...|  carrot_1|[0.2659941, 0.135...|
|s3://one-image-bu...|  carrot_1|[0.3456596, 0.0, ...|
|s3://one-image-bu...|cucumber_1|[1.394345, 0.0170...|
|s3://one-image-bu...|cucumber_1|[1.8686571, 0.162...|
|s3://one-image-bu...|cucumber_1|[1.4106858, 0.017...|
|s3://one-

In [21]:
df_parquet.printSchema()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- path: string (nullable = true)
 |-- label: string (nullable = true)
 |-- features: array (nullable = true)
 |    |-- element: float (containsNull = true)

In [22]:
from pyspark.ml.linalg import Vectors, VectorUDT, DenseVector
transform_vecteur_dense = udf(lambda r: Vectors.dense(r), VectorUDT())
df_parquet= df_parquet.withColumn('features_vectors', transform_vecteur_dense('features'))

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
from pyspark.ml.feature import StandardScaler
scaler_std = StandardScaler(inputCol="features_vectors", outputCol="features_scaled", withStd=True, withMean=True)
model_std = scaler_std.fit(df_parquet)
# Scaled dataframe
df_parquet = model_std.transform(df_parquet)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [24]:
from pyspark.ml.feature import PCA
pca = PCA(k=1000, inputCol='features_vectors', outputCol='pca_features')
model = pca.fit(df_parquet)
#pca_feat_df = model.transform(df_parquet)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
variance=model.explainedVariance
variance.cumsum()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([0.07651515, 0.15268419, 0.22202806, 0.28421775, 0.34231204,
       0.39870558, 0.45074125, 0.50103637, 0.54716584, 0.5920663 ,
       0.63354434, 0.67448292, 0.71157953, 0.74864058, 0.78483198,
       0.819924  , 0.84965612, 0.87895638, 0.90607059, 0.93291613,
       0.95627775, 0.96364734, 0.96765466, 0.97097776, 0.97417911,
       0.9767978 , 0.97881145, 0.98065691, 0.98206066, 0.98330683,
       0.984399  , 0.98534726, 0.98623249, 0.98711752, 0.98797612,
       0.9887302 , 0.9894461 , 0.99001294, 0.99055607, 0.99107029,
       0.99152461, 0.99193966, 0.99235066, 0.9927285 , 0.99309814,
       0.99342654, 0.99375383, 0.994065  , 0.99436239, 0.9946467 ,
       0.99490712, 0.99516137, 0.99540728, 0.99564124, 0.99585642,
       0.99606524, 0.9962599 , 0.99644874, 0.99662559, 0.9967956 ,
       0.99695691, 0.99711055, 0.99726307, 0.99740311, 0.99753749,
       0.99767088, 0.99779247, 0.9979133 , 0.99802922, 0.99814313,
       0.99825246, 0.99835901, 0.99845868, 0.99855453, 0.99865

In [27]:
varCum=variance.cumsum()
val=next(x for x in varCum if x > 0.95)
kInd = np.where(varCum > val)[0][0]#index where cumsum became higher than 0.95

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
#calculate the new PCA features
pca = PCA(k=kInd, inputCol='features_vectors', outputCol='pca_features')
model = pca.fit(df_parquet)
pca_feat_df = model.transform(df_parquet)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
pca_feat_df.write.mode("overwrite").parquet(path_result_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
#Reading results from s3 (format parquet)
if True:
    if True:
        print('lecture distante S3')
        df_parquet = spark.read.format('parquet').load(path_result_pca)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

lecture distante S3

In [31]:
df_parquet.show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|                path|     label|            features|    features_vectors|     features_scaled|        pca_features|
+--------------------+----------+--------------------+--------------------+--------------------+--------------------+
|s3://one-image-bu...|cucumber_1|[2.020552, 0.1632...|[2.02055191993713...|[1.79341598535171...|[-8.2108832148459...|
|s3://one-image-bu...|cucumber_1|[0.676391, 0.1460...|[0.67639100551605...|[-0.1958355573366...|[-9.2669856924734...|
|s3://one-image-bu...|  carrot_1|[0.33251444, 0.37...|[0.33251443505287...|[-0.7047456074275...|[19.5300824357198...|
|s3://one-image-bu...|  carrot_1|[0.3478044, 0.003...|[0.34780439734458...|[-0.6821176759424...|[15.3819217710517...|
|s3://one-image-bu...|  carrot_1|[0.0818258, 0.149...|[0.08182580024003...|[-1.0757449038552...|[17.8158155763371...|
|s3://one-image-bu...|  carrot_1|[0.19739081, 0.12...|[0